## Analysis using ChatGPT
#### Analyzing posts containing configuration errors on various attributes

In this notebook, we use the LLM ChatGPT (chatgpt3.5-turbo) to analyze a set of posts containing configuration errors on different aspects like the tech stack, cause of the configuration error, or impact of the configuration error. For this purpose we will use the 'true'-labeled documents from our manually selected dataset as well as some false positives by the best performing prompting method from the second approach of the first experiment. This will help us to better understand why the false positives resulted at and what we can learn from that.

2023, Ferris Kleier

Loading of libraries and secrets

In [34]:
import requests
import json
import datetime
import secret
import xml.etree.ElementTree as ET
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

API_KEY = secret.get_apikey()
URL = secret.get_url()

#### Function definitions for this notebook

Logging the results

In [39]:
def logging(time, role, outputs, tokens):
    log_string = '\nDate: ' + time + '\nRole: ' + role + '\n\nOutputs: \n\n'
    for output in outputs:
        log_string = log_string + 'Input: #' + str(output[0]) + ', ' + output[1] + '\nOutput:\n' + output[2] + '\n\n'
    log_string = log_string + '\n' + 'Tokens: ' + str(tokens)

    with open("../Code/analysis_log.txt", "a") as f:
        f.write("\n\n--------------------------------")
        f.write(log_string)

In [48]:
def microlog(output, tokens):
    log_string = '\n\nInput: #' + str(output[0]) + ', ' + output[1] + '\nOutput:\n' + output[2] + '\n'
    log_string = log_string + '\n' + 'Tokens: ' + str(tokens)

    with open("../Code/analysis_log.txt", "a") as f:
        f.write(log_string)

The prompting method

In [36]:
def prompt(role, input):
    response = requests.post(URL, headers={'Authorization': API_KEY}, json={
        'messages': [
            {'role': 'system', 'content': role},
            {'role': 'user', 'content': input}
        ], 
        'temperature': 0,
    })
    
    return json.loads(response.text)

In [55]:
with open("../Code/old/output.txt", 'r') as file:
    lines = file.readlines()
    sum_of_numbers = 0
    i = 0
    for line in lines:
        if line.startswith("Tokens: "):
            i = i + 1
            result_string = line[len("Tokens: "):]
            number = int(result_string.strip())
            sum_of_numbers += number
    print(sum_of_numbers)
    print(i)

304133
297


### Dataset creation

Splitting the dataset to only contain the true labeled posts and posts that resulted in false positives by the Few-Shot-Prompting model. We are using the output of the best performing Few-Shot-Prompting model according to it's measurements.

In [37]:
def split():
    false_positives = ["329","332","335","339","350","351","356","363","369","371","379","383","385","393","397","411","416","417","418","426","431","434","435","439","441","442","444","448","460","462","476","481","493","499","500","501"]
    with open("../Posts/dataset.xml", encoding="utf-8-sig") as xml_file:
        dataset = ET.parse(xml_file)
        root = dataset.getroot()
        elements = root.findall("post")

        posts = []
        for element in elements:
            if element.get("label") == "True":
                posts.append([element.get("id"),"True Label",element.get("text")])
            if element.get("id") in false_positives:
                posts.append([element.get("id"),"False Positive",element.get("text")])

    return posts

posts = split()

### Documents Analysis

In this part, we use the API to analyze the dataset of true labeled posts. We constructed this prompt to include all aspects of the configuration error we want to be covered.

In [ ]:
role = "You are given a post from Stack Overflow that contains talk about configuration errors. Analyze the post and it's configuration error \
          according to the following points: \
            1. Tech stack used: What information can be found in the post on the technology used? For example the programming language, frameworks, \
              or databases, is containerization used, is there information on the operating system, version control tools, or network? \
            2. Type of configuration error: What type of configuration error is the post about? Is it a missing configuration parameter, an invalid \
              configuration value, or a conflict between two configuration settings? \
            3. Cause of the configuration error: What caused the configuration error? Was it a typo, a misunderstanding of the configuration \
              documentation, or a bug in the software? \
            4. Impact of the configuration error: What impact did the configuration error have on the software? Did it cause the software to crash, \
              to behave incorrectly, or to produce unexpected results? \
            5. How to fix the configuration error: How can the configuration error be fixed? What tips would you suggest to the user to solve the \
              problem? Keep it short and simple."

time = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
tokens = 0
outputs = []

for post in posts[220:]:
    output = prompt(role, post[2])
    print([post[0],post[1],output['choices'][0]['message']['content']])
    outputs.append([post[0],post[1],output['choices'][0]['message']['content']])
    print(output['usage']['total_tokens'])
    tokens = tokens + output['usage']['total_tokens']
    microlog([post[0],post[1],output['choices'][0]['message']['content']],output['usage']['total_tokens'])

# logging(time,role,outputs,tokens)